# Intensivbetten Status
Eine Auswertung der Auslastung der ICU Kapazitäten mit aufschlüsselung nach Low, High und ECMO care in korrelation zu den Infektionszahlen.

In [28]:
import pandas as pd
import sqlalchemy as db
# Import Database and select necessary data
engine = db.create_engine('sqlite:///data.db')
conn = engine.connect()

# Read Cov19 cases seperated by Landkreis, Refdatum, Geschlecht
betten = pd.read_sql_query(
    "SELECT IdLandkreis, Refdatum, sum(AnzahlFall) as Fälle, SUM(AnzahlGenesen) as Genesen, SUM(AnzahlTodesfall) as Todesfall FROM covid c GROUP BY IdLandkreis, Altersgruppe, Geschlecht ORDER BY Refdatum;", conn)
# Print result
betten


,IdLandkreis,Refdatum,Fälle,Genesen,Todesfall
0,5962,2020-01-02,639,482,129
1,9190,2020-01-10,2450,2219,6
2,9162,2020-01-23,29100,24721,2
3,9181,2020-01-23,1746,1474,0
4,10041,2020-01-23,668,462,171
...,...,...,...,...,...
7486,9775,2022-01-12,1,0,0
7487,9779,2022-01-12,1,0,0
7488,10042,2022-01-12,1,0,0
7489,12070,2022-01-12,1,0,0


In [63]:
import matplotlib.pyplot as plt
import numpy as np

plot = betten.drop(['Genesen', 'Todesfall'], axis=1)
plot = plot.dropna(subset=['Refdatum'])#.iloc[1:2]
# Convert to Date
plot["Refdatum"] = plot["Refdatum"].astype("datetime64")
#print(plot)

ax = list()

# Split into Landkreis
for entity in plot['IdLandkreis']:
    tmp = plot[plot['IdLandkreis']==entity].drop(['IdLandkreis'], axis=1)
    ax.append(tmp)

kwargs = dict(alpha=0.5, bins=100)

for a in ax:
    col = (np.random.random(), np.random.random(), np.random.random())
    #print(a)
    plt.hist(a, **kwargs, color=[col,col], label='Fälle')

plt.gca().set(title='Cov19 Cases', ylabel='Cases')
plt.xlim(50,75)
plt.legend();
